## Langchain 라이브러리의 청킹(Chunking) 관련 클래스와 함수

- CharacterTextSplitter
  -  텍스트를 지정된 기준으로 나눠주는 기본적인 클래스
  -  텍스트가 간단한 구조일 때 적절(잘 정돈된 텍스트, csv 파일 등)
  - 구분자는 디폴트로 "\n\n"을 사용하지만 변경 가능
  - 구분자가 단일 문자이기 때문에, 연속된 텍스트의 의미를 충분히 고려하지 못할 수 있음

- RecursiveCharacterTextSplitter
  - 텍스트를 여러 단계를 거쳐 나누어주는 클래스(청킹에서 가장 많이 활용됨)
  - 텍스트가 상대적으로 복잡한 구조일 때 적절(일반적인 문서, 웹페이지, 보고서 등)
  - 리스트를 통해 구분자를 여러 개 사용하여 세분화 된 청크로 만들 수 있음(구분자의 순서가 중요함)
  - 처음엔 "\n\n"으로 분리하고 청크가 여전히 크다면 "\n", 그래도 크다면 " " 이런식으로 지정된 크기 제한 내에 들어올 때까지 계층적 분할을 진행
  - 계층적 분할을 통해 의미가 있는 단위가 가능한 온전하게 유지되도록 함

**간단한 텍스트에서는 CharacterTextSplitter가 Recursive에 비해 더 속도가 빨라 대용량 처리에 유리할 수 있음**

In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [2]:
text1 = '청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?'
text2 = '청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다.'
text3 = '''청킹을 위한 예제 데이터 입니다.
어떻게 쪼개지는지 알아볼까요?

결과를 확인해봅시다. '''

### 1) Character Splitter

In [11]:
# 청킹 객체 생성

chunk_size = 20      # 하나의 청크에 대한 길이 기준을 문자 20개로 설정
chunk_overlap = 0

c_splitter = CharacterTextSplitter(
    #separator=' ',
    # 청크 크기 지정
    chunk_size = chunk_size,
    # 청크 간 겹치는 부분 크기 지정
    chunk_overlap = chunk_overlap
    )

In [12]:
# split_text : 텍스트 데이터 청킹 실행 함수
c_splitter.split_text(text1)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?']

In [13]:
c_splitter.split_text(text2)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다.']

In [14]:
c_splitter.split_text(text3)

# 무조건 chunk_size보다 작은 크기의 청크로 분리되는 것은 아니고, 의미 있는 문장이라면 splitter의 조건에 따라 더 길게 출력될 수 있음

Created a chunk of size 35, which is longer than the specified 20


['청킹을 위한 예제 데이터 입니다.\n어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

- 발생한 경고 메세지는 지정된 20자 크기 기준보다 더 큰 청크가 생성되었다는 뜻으로 CharacterTextSplitter 의 디폴트인 '\n\n' 이 텍스트 내에서 충분히 자주 발생하지 않아서 20자 이내로 나눌 수 없는 경우 발생됨

### 2) Recursive Splitter

In [15]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
    )

In [16]:
r_splitter.split_text(text1)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?']

In [17]:
r_splitter.split_text(text2)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

In [18]:
r_splitter.split_text(text3)

['청킹을 위한 예제 데이터 입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

### Character, Recursive 비교

In [19]:
see_you_again = """It's been a long day without you, my friend.
And I'll tell you all about it when I see you again.
We've come a long way from where we began.
Oh, I'll tell you all about it when I see you again.
When I see you again.


Damn, who knew?
All the planes we flew, good things we been through.
That I'd be standing right here talking to you.
'Bout another path, I know we loved to hit the road and laugh.
But something told me that it wouldn't last.
Had to switch up, look at things different, see the bigger picture.
Those were the days, hard work forever pays.
Now I see you in a better place (see you in a better place).
Uh."""

In [20]:
len(see_you_again)

620

In [ ]:
# 너 없는 긴 하루였어, 내 친구
# 그리고 너를 다시 만나면 그것에 대해 모두 말해 줄게
# 우리는 우리가 시작한 곳에서 먼 길을 왔고
# 오, 너를 다시 만나면 그것에 대해 모두 말해 줄게
# 널 다시 만나면
#
# 젠장, 누가 알았겠어?
# 우리가 날아간 모든 비행기들, 우리가 겪은 좋은 일들
# 내가 바로 여기 서서 당신과 이야기를 하고 있다는 것,
# 다른 길에 대해, 우리가 길을 가며 웃는 것을 좋아했다는 것을 알아요
# 하지만 어떤 것이 그것이 지속되지 않을 것이라고 말했어요
# 일어나, 다른 것을 봐, 더 큰 그림을 봐
# 그 시절은 좋았어, 노력은 영원히 보상받을 거야
# 이제 더 나은 곳에서 만나 (더 나은 곳에서 만나)
# Uh

In [21]:
# separator로 청크로 나누는 기준을 설정할 수 있음
c_splitter = CharacterTextSplitter(chunk_size=100,
                                   chunk_overlap=0,
                                   separator = '\n\n'    # 직접 지정 가능
                                  )
c_splitter.split_text(see_you_again)

Created a chunk of size 215, which is longer than the specified 100


["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.\nWe've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.\nWhen I see you again.",
 "Damn, who knew?\nAll the planes we flew, good things we been through.\nThat I'd be standing right here talking to you.\n'Bout another path, I know we loved to hit the road and laugh.\nBut something told me that it wouldn't last.\nHad to switch up, look at things different, see the bigger picture.\nThose were the days, hard work forever pays.\nNow I see you in a better place (see you in a better place).\nUh."]

In [22]:
r_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                            chunk_overlap=0,
                                            separators=["\n\n", "\n", " "]   # 리스트로 계층적 조건을 설정 가능
                                           )
r_splitter.split_text(see_you_again)

["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.",
 "We've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.",
 'When I see you again.',
 'Damn, who knew?\nAll the planes we flew, good things we been through.',
 "That I'd be standing right here talking to you.",
 "'Bout another path, I know we loved to hit the road and laugh.",
 "But something told me that it wouldn't last.",
 'Had to switch up, look at things different, see the bigger picture.',
 'Those were the days, hard work forever pays.',
 'Now I see you in a better place (see you in a better place).\nUh.']

- CharacterTextSplitter를 사용한 결과가 만족스럽지 못하다면 더 작은 단위의 구분자를 지정("\n", " ")하여 해결하거나 RecursiveCharacterTextSplitter를 사용하여 계층적 분할을 적용하면 됨
- 즉 CharacterTextSplitter보다 RecursiveCharacterTextSplitter가 더욱 사용하기 편리하고 결과가 좋음

## 웹 페이지에 접근해 쉽게 텍스트 데이터 가져오기

#### LangChain(2022.10)
- 자연어 처리(NLP)와 사전 학습된 LLM모델을 사용하여 다양한 응용프로그램을 구축할 수 있게 해주는 라이브러리(LangSmith로 설계 및 배포기능까지 지원)
- 데이터베이스, API, 외부 서비스(ChatGPT 등)와의 통합을 지원하는 인터페이스 제공(openai 외에도 Anthropic, Hugging Face, Google 등 다양한 기업들의 LLM 모델들과 연동)
- RAG 구현을 위한 기본적인 인터페이스와 임베딩, Advanced RAG 등의 기능을 제공
- 공식 페이지 : https://www.langchain.com/
- langchain docs 페이지 : https://python.langchain.com/v0.2/docs/introduction/

In [ ]:
# langchain 설치
!pip install -U langchain

In [ ]:
# langchain과 openai의 GPT모델을 결합한 기능을 이용
!pip install -U langchain-openai

In [ ]:
# langchain 생태계를 확장하고 기본 langchain기능을 보완하는 다양한 도구와의 통합을 제공
!pip install -U langchain_community

In [23]:
# WebBaseLoader : langchain라이브러리에서 지원하는 웹 페이지 HTML 콘텐츠를 로드해주는 클래스
#                 (내부적으로 BeautifulSoup을 사용함)
from langchain.document_loaders import WebBaseLoader

# WARNING은 웹 접근시 user_agent 설정이 필요하다는 경고 문구

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [25]:
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/003/0012627594")

# SSL 인증 오류 우회 설정
# loader.requests_kwargs = {'verify': False}

# 브라우저 우회 접속
loader.requests_kwargs = {'headers': {'User-Agent': 'Mozilla/5.0'}}   # 개발자도구 - Network - 요소 클릭후 우측 Headers탭 확인
# 한글 인코딩 설정
loader.encoding = 'utf-8'

docs = loader.load()

In [29]:
docs[0].page_content[:500]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\nNAVER\n\n뉴스\n\n\n엔터\n\n\n\n\n스포츠\n\n\n\n\n날씨\n\n\n\n\n프리미엄\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n뉴시스\n\n뉴시스\n\n\n구독\n\n뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴시스 언론사 구독 해지되었습니다.\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은\n\n\n\n\n입력2024.06.25. 오후 2:49\n\n\n수정2024.06.25. 오후 6:01\n\n기사원문\n \n\n\n\n\n박은비 기자\n\n\n\n\n\n\n\n\n박은비 '

In [27]:
print(docs[0].page_content[:500])




















"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은














본문 바로가기






NAVER

뉴스


엔터




스포츠




날씨




프리미엄










검색










언론사별


정치


경제


사회


생활/문화


IT/과학


세계


랭킹


신문보기


오피니언


TV


팩트체크


알고리즘 안내


정정보도 모음
















뉴시스

뉴시스


구독

뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.
보러가기


뉴시스 언론사 구독 해지되었습니다.




PICK
안내


언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기




"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은




입력2024.06.25. 오후 2:49


수정2024.06.25. 오후 6:01

기사원문
 




박은비 기자








박은비 


- 문자열 처리 코드나 정규표현식 등을 활용하여 필요한 텍스트만 추출해서 사용할 수 있음

### 텍스트 정제
#### 1) 문자열 함수 replace 활용

In [25]:
docs[0].page_content[:500].replace("\n", "")

'"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은본문 바로가기NAVER뉴스엔터스포츠날씨프리미엄검색언론사별정치경제사회생활/문화IT/과학세계랭킹신문보기오피니언TV팩트체크알고리즘 안내정정보도 모음뉴시스뉴시스구독뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.보러가기뉴시스 언론사 구독 해지되었습니다.PICK안내언론사가 주요기사로선정한 기사입니다.언론사별 바로가기닫기"거품이었나"…시총 앞자리 바뀐 엔비디아, 주가 변곡점은입력2024.06.25. 오후 2:49수정2024.06.25. 오후 6:01기사원문 박은비 기자박은비 '

#### 2) 문자열 정규표현식 활용
- 정규표현식은 검색 결과가 개별 문자열로 출력되기 때문에 필요에 따라 이를 모두 합쳐서 텍스트로 활용해야 함

In [26]:
import re

In [27]:
# 문자열 앞 r 의 의미는 문자열 내에 이스케이프 코드를 지칭하는 \(역슬래시)가 있을 경우 이를 일반 문자열로 인지하게 함
# 가-힣 : 한글 문자열 전체
# a-zA-Z : 영문 문자열(대,소문자 포함) 전체
# 0-9_ : 숫자 및 언더바(_) 전체
# + : 연속된 문자열 반환(띄워쓰기 기준)
re.findall(r"[\w]+", docs[0].page_content[:500])   # [\w]+ 작성시 리터럴 경고가 발생하여 앞에 r 붙여줄 것

['거품이었나',
 '시총',
 '앞자리',
 '바뀐',
 '엔비디아',
 '주가',
 '변곡점은',
 '본문',
 '바로가기',
 'NAVER',
 '뉴스',
 '엔터',
 '스포츠',
 '날씨',
 '프리미엄',
 '검색',
 '언론사별',
 '정치',
 '경제',
 '사회',
 '생활',
 '문화',
 'IT',
 '과학',
 '세계',
 '랭킹',
 '신문보기',
 '오피니언',
 'TV',
 '팩트체크',
 '알고리즘',
 '안내',
 '정정보도',
 '모음',
 '뉴시스',
 '뉴시스',
 '구독',
 '뉴시스',
 '언론사',
 '구독되었습니다',
 '메인',
 '뉴스판에서',
 '주요뉴스를',
 '볼',
 '수',
 '있습니다',
 '보러가기',
 '뉴시스',
 '언론사',
 '구독',
 '해지되었습니다',
 'PICK',
 '안내',
 '언론사가',
 '주요기사로선정한',
 '기사입니다',
 '언론사별',
 '바로가기',
 '닫기',
 '거품이었나',
 '시총',
 '앞자리',
 '바뀐',
 '엔비디아',
 '주가',
 '변곡점은',
 '입력2024',
 '06',
 '25',
 '오후',
 '2',
 '49',
 '수정2024',
 '06',
 '25',
 '오후',
 '6',
 '01',
 '기사원문',
 '박은비',
 '기자',
 '박은비']

In [28]:
" ".join(re.findall(r"[\w]+", docs[0].page_content))

'거품이었나 시총 앞자리 바뀐 엔비디아 주가 변곡점은 본문 바로가기 NAVER 뉴스 엔터 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활 문화 IT 과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 뉴시스 뉴시스 구독 뉴시스 언론사 구독되었습니다 메인 뉴스판에서 주요뉴스를 볼 수 있습니다 보러가기 뉴시스 언론사 구독 해지되었습니다 PICK 안내 언론사가 주요기사로선정한 기사입니다 언론사별 바로가기 닫기 거품이었나 시총 앞자리 바뀐 엔비디아 주가 변곡점은 입력2024 06 25 오후 2 49 수정2024 06 25 오후 6 01 기사원문 박은비 기자 박은비 기자 박은비 기자 구독 구독중 구독자 0 응원수 0 더보기 추천 쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0 댓글 본문 요약봇 본문 요약봇도움말 자동 추출 기술로 요약된 내용입니다 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다 닫기 텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름 이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다 본문듣기 시작 닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기 상당부분 선반영 추가 상승 모멘텀 관건 26일 있을 주총 마이크론 실적 발표 주목 서울 뉴시스 젠슨 황 엔비디아 최고경영자 CEO 2024 06 03 photo newsis com 재판매 및 DB 금지 서울 뉴시스 박은비 기자 미국 반도체 기업 엔비디아 주가가 3거래일 만에 13 가까이 급락하면서 달리는 주가에 올라탔던 개인투자자들이 아우성이다 고대역폭메모리 HBM 시장을 장악하고 있는 엔비디아의 시장지배력을 토대로 낙관론이 우세하지만 인공지능 AI 열풍 과열론도 고개를 들고 있다 이 가운데 곧 있을 주주총회와 동종업계 마이크론 실적 발표 등을 통해 상승 추세가